In [ ]:
import cv2
import dlib
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
# Create output directory if not exists
output_folder = "extracted_faces"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Load image
def detect_and_save_faces(image_path):
    detector = dlib.get_frontal_face_detector()
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    face_images = []
    for i, face in enumerate(faces):
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        face_img = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face_img, (100, 100))
        face_path = os.path.join(output_folder, f"face_{i}.jpg")
        cv2.imwrite(face_path, face_resized)
        face_images.append(face_resized)

    return face_images

In [ ]:
# Train a simple CNN on extracted faces
def build_face_recognition_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 1)),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')  # Assuming two classes for demo
    ])
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Prepare dataset for training
def prepare_data():
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_data = datagen.flow_from_directory(output_folder, target_size=(100, 100), color_mode='grayscale', batch_size=10, class_mode='sparse', subset='training')
    val_data = datagen.flow_from_directory(output_folder, target_size=(100, 100), color_mode='grayscale', batch_size=10, class_mode='sparse', subset='validation')
    return train_data, val_data

if __name__ == "__main__":
    # Change this to your actual group photos directory
    group_photos_dir = "/content/drive/MyDrive/Group Photos"

    # Loop through all files in the directory
    for filename in os.listdir(group_photos_dir):
        if filename.endswith(('.jpg', '.png', '.jpeg')):  # Check if it's an image file
            image_path = os.path.join(group_photos_dir, filename)
            detect_and_save_faces(image_path)

    model = build_face_recognition_model()
    train_data, val_data = prepare_data()

    model.fit(train_data, validation_data=val_data, epochs=10)
    model.save("face_recognition_model.h5")
    print("Model trained and saved successfully.")